# Pt 2: Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [24]:
# Import dependencies
import pandas as pd
import gmaps
import requests

from config import g_key

In [25]:
# Open File
weather_data_df = pd.read_csv("challenge_data/WeatherPy_challenge.csv")
weather_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Khatanga,RU,2020-02-23 18:27:20,71.97,102.50,12.67,92,98,15.46,light snow,0.00,0.25
1,1,Sao Filipe,CV,2020-02-23 18:27:20,14.90,-24.50,68.52,83,100,18.84,overcast clouds,0.00,0.00
2,2,Albany,US,2020-02-23 18:27:21,42.60,-73.97,51.01,29,75,4.70,broken clouds,0.00,0.00
3,3,Bahir Dar,ET,2020-02-23 18:26:17,11.59,37.39,75.31,25,35,7.09,scattered clouds,0.00,0.00
4,4,Butaritari,KI,2020-02-23 18:25:37,3.07,172.79,84.70,76,100,17.56,moderate rain,6.37,0.00
5,5,Milkovo,RU,2020-02-23 18:21:05,54.72,158.62,-4.72,99,100,0.45,light snow,0.00,0.19
6,6,Ambanja,MG,2020-02-23 18:27:21,-13.68,48.45,78.33,84,100,4.65,light rain,0.19,0.00
7,7,Matara,LK,2020-02-23 18:27:21,5.95,80.54,75.51,87,94,10.89,overcast clouds,0.00,0.00
8,8,Provideniya,RU,2020-02-23 18:27:22,64.38,-173.30,-9.76,73,0,2.24,clear sky,0.00,0.00
9,9,Hilo,US,2020-02-23 18:27:22,19.73,-155.09,70.00,64,40,4.70,scattered clouds,0.00,0.00


In [57]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is your minimum temperature preference? "))
max_temp = float(input("What is your maximum temperature preference? "))

# Ask the customer if they would like it to be raining or not
want_rain = str(input("Do you want it to be raining? (yes/no) "))
# Ask the customer if they would like it to be snowing or not
want_snow = str(input("Do you want it to be snowing? (yes/no) "))

What is your minimum temperature preference? 60
What is your maximum temperature preference? 80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [58]:
# Filter the dataset to find the cities that fit the criteria
if want_rain == "yes" and want_snow == "yes":
    preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                              (weather_data_df["Max Temp"] >= min_temp) &
                                              (weather_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                              (weather_data_df["Snow inches (last 3 hrs)"] > 0)]

elif want_rain == "yes" and want_snow == "no":
    preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                              (weather_data_df["Max Temp"] >= min_temp) &
                                              (weather_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                              (weather_data_df["Snow inches (last 3 hrs)"] == 0)]

elif want_rain == "no" and want_snow == "yes":
    preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) &
                                              (weather_data_df["Max Temp"] >= min_temp) &
                                              (weather_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                              (weather_data_df["Snow inches (last 3 hrs)"] > 0)]

else:
    preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                              (weather_data_df["Max Temp"] >= min_temp) &
                                              (weather_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                              (weather_data_df["Snow inches (last 3 hrs)"] == 0)]
    
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
1,1,Sao Filipe,CV,2020-02-23 18:27:20,14.90,-24.50,68.52,83,100,18.84,overcast clouds,0.0,0.0
3,3,Bahir Dar,ET,2020-02-23 18:26:17,11.59,37.39,75.31,25,35,7.09,scattered clouds,0.0,0.0
7,7,Matara,LK,2020-02-23 18:27:21,5.95,80.54,75.51,87,94,10.89,overcast clouds,0.0,0.0
9,9,Hilo,US,2020-02-23 18:27:22,19.73,-155.09,70.00,64,40,4.70,scattered clouds,0.0,0.0
10,10,Vaini,TO,2020-02-23 18:27:22,-21.20,-175.20,77.00,100,75,6.93,broken clouds,0.0,0.0
17,17,Saldanha,ZA,2020-02-23 18:25:53,-33.01,17.94,78.80,34,0,12.75,clear sky,0.0,0.0
25,25,Kapaa,US,2020-02-23 18:27:25,22.08,-159.32,69.80,82,1,9.17,clear sky,0.0,0.0
33,33,Port Elizabeth,ZA,2020-02-23 18:26:31,-33.92,25.57,62.60,72,8,3.36,clear sky,0.0,0.0
35,35,Saint-Philippe,RE,2020-02-23 18:27:02,-21.36,55.77,77.00,78,0,3.36,clear sky,0.0,0.0
36,36,Rio Gallegos,AR,2020-02-23 18:27:29,-51.62,-69.22,62.60,36,40,16.11,scattered clouds,0.0,0.0


In [59]:
# Create a DataFrame called hotel_df to store hotel names along with 
# city, country, max temp, current weather description, latitude, longitude 
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Filipe,CV,68.52,overcast clouds,14.90,-24.50,
3,Bahir Dar,ET,75.31,scattered clouds,11.59,37.39,
7,Matara,LK,75.51,overcast clouds,5.95,80.54,
9,Hilo,US,70.00,scattered clouds,19.73,-155.09,
10,Vaini,TO,77.00,broken clouds,-21.20,-175.20,
17,Saldanha,ZA,78.80,clear sky,-33.01,17.94,
25,Kapaa,US,69.80,clear sky,22.08,-159.32,
33,Port Elizabeth,ZA,62.60,clear sky,-33.92,25.57,
35,Saint-Philippe,RE,77.00,clear sky,-21.36,55.77,
36,Rio Gallegos,AR,62.60,scattered clouds,-51.62,-69.22,


In [60]:
# Configure gmaps to use my Google API Key
gmaps.configure(api_key=g_key)

In [61]:
# Set the parameters for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [62]:
# Iterature through the Hotel DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # Use the search term "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store it
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [63]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Filipe,CV,68.52,overcast clouds,14.90,-24.50,Tortuga B&B
3,Bahir Dar,ET,75.31,scattered clouds,11.59,37.39,"Delano Hotel, Bahir Dar"
7,Matara,LK,75.51,overcast clouds,5.95,80.54,Amaloh Boutique Resort
9,Hilo,US,70.00,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
10,Vaini,TO,77.00,broken clouds,-21.20,-175.20,Keleti Beach Resort
17,Saldanha,ZA,78.80,clear sky,-33.01,17.94,Blue Bay Lodge
25,Kapaa,US,69.80,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
33,Port Elizabeth,ZA,62.60,clear sky,-33.92,25.57,39 On Nile Guest House
35,Saint-Philippe,RE,77.00,clear sky,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
36,Rio Gallegos,AR,62.60,scattered clouds,-51.62,-69.22,Hotel Santa Cruz


In [69]:
# Create the output file (CSV)
output_data_file = "challenge_data/WeatherPy_vacation.csv"

# Export the hotel DataFrame into a CSV file
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [70]:
# Description Definition
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [71]:
# Add marker layer map
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))